그리드 서치는 하이퍼파라미터 값에 대한 최적의 조합을 찾는 튜닝 기법이다.</br>
그리스 서치는 리스트로 지정된 값의 모든 조합에 대해서 성능을 평가해서 최적의 조합을 찾는다.</br>
가능한 모든 조합을 찾기때문에 계산 비용이 많이 든다.</br>

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv(   # 위스콘신 유방암 데이터셋 로딩
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', 
    header=None
)

x = df.loc[:, 2:].values    # 30개의 특성을 x에 할당
y = df.loc[:, 1].values     # 클래스 레이블을 y에 할당
y = LabelEncoder().fit_transform(y) # 클래스 레이블을 정수로 변환(악성: 1, 양성: 0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(
    StandardScaler(), 
    SVC(random_state=1)     # 서포트벡터머신을 사용해서 훈련
)
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [  # 튜닝 대상 매개변수의 리스트
    # 선형 SVM은 규제 매개변수 C만 튜닝한다.
    {'svc__C': param_range, 'svc__kernel': ['linear']},
    # RBF 커널 SVM에서는 C와 gamma 매개변수를 튜닝한다.
    {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']}
]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=1)
gs = gs.fit(x_train, y_train)

print(gs.best_score_)   # 최상의 모델 점수
print(gs.best_params_)  # 최상의 모델 점수를 만든 매개변수 값

clf = gs.best_estimator_    # 그리드서치에 의해서 선택된 최고의 모델을 할당한다.
clf.fit(x_train, y_train)   # 최고의 모델로 훈련셋을 훈련한다.

# 테스트셋을 사용해서 모델의 성능 측정
print('테스트 정확도: %.3f' % clf.score(x_test, y_test))


0.9846859903381642
{'svc__C': 100.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
테스트 정확도: 0.974
